In [1]:
import pandas as pd
import numpy as np
import os
import scipy
from scipy.interpolate import griddata
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.nn.functional import mse_loss
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import glob
from compute_distance import compute_distance

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
print(f"Using device: {device}")

h_nD = 30
h_n = 20
input_n = 4
learning_rate = 5e-4

class Swish(nn.Module):
    def __init__(self, inplace=True):
        super(Swish, self).__init__()
        self.inplace = inplace

    def forward(self, x):
        if self.inplace:
            x.mul_(torch.sigmoid(x))
            return x
        else:
            return x * torch.sigmoid(x)


class Net1(nn.Module):

    #The __init__ function stack the layers of the 
    #network Sequentially 
    def __init__(self):
        super(Net1, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_n,h_n),
            #nn.Tanh(),
            #nn.Sigmoid(),
            Swish(),
            nn.Linear(h_n,h_n),
            #nn.Tanh(),
            #nn.Sigmoid(),
            Swish(),
            nn.Linear(h_n,h_n),
            #nn.Tanh(),
            #nn.Sigmoid(),
            Swish(),

            nn.Linear(h_n,1),
        )
    #This function defines the forward rule of
    #output respect to input.
    def forward(self,x):
        output = self.main(x)
        return  output

class Net2(nn.Module):

    #The __init__ function stack the layers of the 
    #network Sequentially 
    def __init__(self):
        super(Net2, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_n,h_n),
            #nn.Tanh(),
            #nn.Sigmoid(),
            Swish(),
            nn.Linear(h_n,h_n),
            #nn.Tanh(),
            #nn.Sigmoid(),
            Swish(),
            nn.Linear(h_n,h_n),
            #nn.Tanh(),
            #nn.Sigmoid(),
            Swish(),

            nn.Linear(h_n,1),
        )
    #This function defines the forward rule of
    #output respect to input.
    def forward(self,x):
        output = self.main(x)
        return  output

class Net3(nn.Module):

    #The __init__ function stack the layers of the 
    #network Sequentially 
    def __init__(self):
        super(Net3, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_n,h_n),
            #nn.Tanh(),
            #nn.Sigmoid(),
            Swish(),
            nn.Linear(h_n,h_n),
            #nn.Tanh(),
            #nn.Sigmoid(),
            Swish(),
            nn.Linear(h_n,h_n),
            #nn.Tanh(),
            #nn.Sigmoid(),
            Swish(),
            ################## below are added layers

            nn.Linear(h_n,1),
        )
    #This function defines the forward rule of
    #output respect to input.
    def forward(self,x):
        output = self.main(x)
        return  output

Using device: cuda


In [3]:
#Dataset
class CustomDataset(Dataset):
    def __init__(self, data):
        self.inputs = torch.tensor(data[:, [0, 1, 5, 7]], dtype=torch.float32).to(device)
        self.targets = torch.tensor(data[:, [2, 3, 4]], dtype=torch.float32).to(device)


    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]


In [4]:
# Define absolute paths for saved weights
weights_dir = r"models\original"  # Use raw string (r"...") to avoid escape character issues
e_idx = -1
num_epochs = 500
net1_path = os.path.join(weights_dir, f"original_1_epoch_{e_idx}.pth")
net2_path = os.path.join(weights_dir, f"original_2_epoch_{e_idx}.pth")
net3_path = os.path.join(weights_dir, f"original_3_epoch_{e_idx}.pth")

In [5]:
def initialize_models_lbfgs():
    # Move models to GPU
    net1 = Net1().to(device)
    net2 = Net2().to(device)
    net3 = Net3().to(device)

    print(f"net1 is on: {next(net1.parameters()).device}")
    print(f"net2 is on: {next(net2.parameters()).device}")
    print(f"net3 is on: {next(net3.parameters()).device}")

    def init_normal(m):
        if isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight)

    # Initialize weights
    net1.apply(init_normal)
    net2.apply(init_normal)
    net3.apply(init_normal)

    # Load saved weights (if any)
    if os.path.exists(net1_path):
        net1.load_state_dict(torch.load(net1_path))
        net1.train()
        print(f"✅ Loaded weights from {net1_path}")
    else:
        print(f"⚠️ No saved weights found at {net1_path}. Training from scratch.")

    if os.path.exists(net2_path):
        net2.load_state_dict(torch.load(net2_path))
        net2.train()
        print(f"✅ Loaded weights from {net2_path}")
    else:
        print(f"⚠️ No saved weights found at {net2_path}. Training from scratch.")

    if os.path.exists(net3_path):
        net3.load_state_dict(torch.load(net3_path))
        net3.train()
        print(f"✅ Loaded weights from {net3_path}")
    else:
        print(f"⚠️ No saved weights found at {net3_path}. Training from scratch.")

    # ✅ Combine all parameters into one LBFGS optimizer
    optimizer = optim.LBFGS(
        list(net1.parameters()) + list(net2.parameters()) + list(net3.parameters()),
        lr=1.0,
        max_iter=50,
        history_size=50,
        tolerance_grad=1e-7,
        line_search_fn='strong_wolfe'
    )

    return net1, net2, net3, optimizer


In [6]:
def normalized_relative_mse(pred, target, scale=1.0, eps=1e-6):
    """
    Computes mean squared relative error, scaled for balance across loss terms.
    """
    relative_error = (pred - target) / (target.abs() + eps)
    return torch.mean(relative_error ** 2) / scale

def criterion(x_batch, y_batch, model_1, model_2, model_3, mse_loss, v_loss_weight, p_loss_weight, rho=1.0, nu=0.01):
    """
    Computes the loss based on physics-based PDE constraints and data loss.

    Args:
    - x_batch: Input tensor (batch of input values)
    - y_batch: Target tensor (batch of true output values)
    - model_1, model_2, model_3: Neural network models for u, v, and p
    - mse_loss: MSE loss function
    - rho: Density parameter for PDE constraints
    - nu: Viscosity parameter for PDE constraints

    Returns:
    - Total loss: Sum of physics loss and data loss
    """
    # Compute model predictions inside criterion
    x, y, d, n = x_batch[:, 0:1], x_batch[:, 1:2], x_batch[:, 2:3], x_batch[:, 3:4]
    
    x.requires_grad_(True)
    y.requires_grad_(True)
    d.requires_grad_(True)
    n.requires_grad_(True)

    model_inputs = torch.cat((x, y, d, n), dim=1)  # Concatenate inputs for the models

    # some sort of ways to actually define the pairs inside of these 50000 points.
    u = model_1(model_inputs)
    v = model_2(model_inputs)
    p = model_3(model_inputs)
    #print(u.shape)
    u = u.view(len(u),-1)
    v = v.view(len(v),-1)
    p = p.view(len(p),-1)
    #print(u.shape)
    
    # Compute distance to the nearest circular post
    distances = compute_distance(x, y, d, n)
    u[distances == 0] = 0
    v[distances == 0] = 0
    v[x == 0] = 0  #inlet y velocity
    p[x == 0.4] = 0
    u_loss = mse_loss(u, y_batch[:, 0:1])
    v_loss = mse_loss(v, y_batch[:, 1:2])
    p_loss = mse_loss(p, y_batch[:, 2:3])


    # Compute data loss (MSE against target values)
    L_data = u_loss + v_loss + p_loss
    #L_data_periodic = ( u_p_loss + v_loss * 500 + p_loss )

    # l_data_periodic + L


    return L_data, u_loss, v_loss, p_loss


In [7]:
def main_lbfgs():
    from torch.nn import MSELoss
    import matplotlib.pyplot as plt

    # Load all training data (full-batch)
    csv_dir   = os.path.join(os.getcwd(), "data", "unscaled")
    pattern   = os.path.join(csv_dir, "W_*_*_1.csv")
    csv_paths = sorted(glob.glob(pattern))

    if not csv_paths:
        raise FileNotFoundError(f"No files matched: {pattern}")

    frames = [pd.read_csv(p) for p in csv_paths]
    df     = pd.concat(frames, ignore_index=True)
    data   = df.to_numpy(dtype=np.float32)

    dataset = CustomDataset(data)
    x_batch, y_batch = dataset[:]  # full batch
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)

    # Load models
    net1, net2, net3, optimizer = initialize_models_lbfgs()
    mse_loss = MSELoss()
    start_time = time.time()

    # Logging storage
    loss_history = {
        'epoch': [],
        'total_loss': [],
        'u_loss': [],
        'v_loss': [],
        'p_loss': []
    }

    v_loss_weight = 1  # Initial weight for v_loss
    p_loss_weight = 1
    min_loss = float('inf')

    latest_losses = {
        'total': 0, 
        'u': 0, 
        'v': 0, 
        'p': 0
    }

    def closure():
        optimizer.zero_grad()
        loss, u_loss, v_loss, p_loss = criterion(
            x_batch, y_batch, net1, net2, net3, mse_loss, v_loss_weight, p_loss_weight)
        loss.backward()

        # Store latest breakdown for logging
        latest_losses['total'] = loss
        latest_losses['u'] = u_loss
        latest_losses['v'] = v_loss
        latest_losses['p'] = p_loss

        return loss

    for epoch in range(num_epochs+1):
        # if epoch > 100:
        #     v_loss_weight = 50

        # if epoch > 200:
        #     v_loss_weight = 100
        
        # if epoch > 300:
        #     v_loss_weight = 200

        # if epoch > 400:
        #     v_loss_weight = 500

        optimizer.step(closure)

        loss = latest_losses['total'].item()
        u_loss = latest_losses['u'].item()
        v_loss = latest_losses['v'].item()
        p_loss = latest_losses['p'].item()

        loss_history['epoch'].append(epoch)
        loss_history['total_loss'].append(loss)
        loss_history['u_loss'].append(u_loss)
        loss_history['v_loss'].append(v_loss)
        loss_history['p_loss'].append(p_loss)

        elapsed_time = time.time() - start_time
        print(f"[{epoch}/{num_epochs}] Total Loss: {loss:.7f} | u_loss: {u_loss:.7f}, v_loss: {v_loss:.7f}, p_loss: {p_loss:.7f} | Time: {elapsed_time:.2f}s")
        start_time = time.time()

        if loss < min_loss:
            min_loss = loss
            torch.save(net1.state_dict(), f'models/original/original_1_epoch_{epoch}.pth')
            torch.save(net2.state_dict(), f'models/original/original_2_epoch_{epoch}.pth')
            torch.save(net3.state_dict(), f'models/original/original_3_epoch_{epoch}.pth')
            print(f"Epoch {epoch}: New minimum loss {min_loss:.7f}, models saved")

        elif epoch % 100 == 0:
            min_loss = loss.item()
            # Save models every 100 epochs
            torch.save(net1.state_dict(), f'models/original/original_1_epoch_{epoch}.pth')
            torch.save(net2.state_dict(), f'models/original/original_2_epoch_{epoch}.pth')
            torch.save(net3.state_dict(), f'models/original/original_3_epoch_{epoch}.pth')
            print(f"Epoch {epoch}: Models saved")
            
    # Save loss log
    os.makedirs("results/original", exist_ok=True)
    loss_df = pd.DataFrame(loss_history)
    loss_df.to_csv("results/original/losses_lbfgs.csv", index=False)

    # Save loss plot
    plt.figure(figsize=(12, 8))
    for k in loss_history:
        if k != 'epoch':
            plt.plot(loss_history['epoch'], loss_history[k], label=k)
    plt.yscale('log')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.title("LBFGS Losses (Log Scale)")
    plt.legend()
    plt.tight_layout()
    plt.savefig("results/original/loss_curve_lbfgs.png")
    plt.show()

    return net1, net2, net3

In [8]:
model_1, model_2, model_3 = main_lbfgs()

net1 is on: cuda:0
net2 is on: cuda:0
net3 is on: cuda:0
⚠️ No saved weights found at models\original\original_1_epoch_-1.pth. Training from scratch.
⚠️ No saved weights found at models\original\original_2_epoch_-1.pth. Training from scratch.
⚠️ No saved weights found at models\original\original_3_epoch_-1.pth. Training from scratch.
[0/500] Total Loss: 0.0059951 | u_loss: 0.0016425, v_loss: 0.0034388, p_loss: 0.0009137 | Time: 12.38s
Epoch 0: New minimum loss 0.0059951, models saved
[1/500] Total Loss: 0.0014607 | u_loss: 0.0009891, v_loss: 0.0002020, p_loss: 0.0002697 | Time: 12.51s
Epoch 1: New minimum loss 0.0014607, models saved
[2/500] Total Loss: 0.0010131 | u_loss: 0.0008324, v_loss: 0.0000990, p_loss: 0.0000817 | Time: 12.24s
Epoch 2: New minimum loss 0.0010131, models saved
[3/500] Total Loss: 0.0008982 | u_loss: 0.0007297, v_loss: 0.0000769, p_loss: 0.0000916 | Time: 12.90s
Epoch 3: New minimum loss 0.0008982, models saved
[4/500] Total Loss: 0.0008473 | u_loss: 0.0006983, v

KeyboardInterrupt: 

In [ ]:
model_1.eval()
model_2.eval()
model_3.eval()

In [ ]:
# PREDICTION USING HARD ENFORCEMENT

# File path
file_path = "data/scaled/DRPINN_0.55_14_1.csv"

# Load CSV file
df = pd.read_csv(file_path)

# Ensure the correct columns are selected for input
input_data = df.iloc[:, [0, 1, 5, 7]].values  # Selecting columns (x, y, d, N)
output_data = df.iloc[:, [2, 3, 4]].values  # Selecting columns (u, v, p)

# Convert to PyTorch tensor and move to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_tensor = torch.tensor(input_data, dtype=torch.float32).to(device)

# Make predictions
with torch.no_grad():
    u_pred = model_1(input_tensor)
    v_pred = model_2(input_tensor)
    p_pred = model_3(input_tensor)

# Extract individual elements
x, y, d, n = input_tensor[:, 0], input_tensor[:, 1], input_tensor[:, 2], input_tensor[:, 3]

# Apply conditions
v_pred[x == 0] = 0  # If x = 0, set v_pred to zero

# Compute distance
distances = compute_distance(x, y, d, n)

# If distance is zero, set u and v to zero
mask = distances == 0
u_pred[mask] = 0
v_pred[mask] = 0

# Move predictions to CPU and convert to NumPy
u_pred = u_pred.cpu().numpy()
v_pred = v_pred.cpu().numpy()
p_pred = p_pred.cpu().numpy()


In [ ]:
#prediction for NO MODIFICATION
import pandas as pd
import torch
import numpy as np

# File path
file_path = "data/scaled/W_0.55_14_1.csv"

# Load CSV file
df = pd.read_csv(file_path)

# Ensure the correct columns are selected for input (same format as training)
input_data = df.iloc[:, [0, 1, 5, 7]].values  # Selecting columns (x, y, d)
output_data = df.iloc[:, [2, 3, 4]].values  # Selecting columns (x, y, d)

# Convert to PyTorch tensor and move to GPU (if available)
input_tensor = torch.tensor(input_data, dtype=torch.float32).to(device)

with torch.no_grad():
    u_pred = model_1(input_tensor)
    v_pred = model_2(input_tensor)
    p_pred = model_3(input_tensor)

# Move predictions to CPU and convert to NumPy
u_pred = u_pred.cpu().numpy()
v_pred = v_pred.cpu().numpy()
p_pred = p_pred.cpu().numpy()


In [ ]:
x_grid = input_data[:, 0].reshape(-1)
y_grid = input_data[:, 1].reshape(-1)
v_actual_grid = output_data[:, 1].reshape(-1)
v_pred_grid = v_pred.flatten()
u_actual_grid = output_data[:, 0].reshape(-1)
u_pred_grid = u_pred.flatten()

In [ ]:
# Create a DataFrame
df = pd.DataFrame({
    'x': x_grid,
    'y': y_grid,
    'u': u_pred_grid,
    'v': v_pred_grid
})

# Save to CSV
df.to_csv('results/original/csv/O_0.55_14.csv', index=False)

print("CSV file saved successfully!")

In [ ]:
v_pred_grid.shape

In [ ]:
%matplotlib inline

# Determine the common range for colorbar
vmin = min(v_actual_grid.min(), v_pred_grid.min())
vmax = max(v_actual_grid.max(), v_pred_grid.max())

# Plotting
plt.figure(figsize=(12, 6))

# Actual u-velocity
plt.subplot(1, 2, 1)
plt.tricontourf(x_grid, y_grid, v_actual_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Actual u-velocity')
plt.xlabel('x')
plt.ylabel('y')

# Predicted u-velocity
plt.subplot(1, 2, 2)
plt.tricontourf(x_grid, y_grid, v_pred_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Predicted u-velocity')
plt.xlabel('x')
plt.ylabel('y')

plt.tight_layout()
plt.show()


In [ ]:
%matplotlib inline

# Determine the common range for colorbar
vmin = min(v_actual_grid.min(), v_pred_grid.min())
vmax = max(v_actual_grid.max(), v_pred_grid.max())

# Plotting
plt.figure(figsize=(12, 6))

# Actual u-velocity
plt.subplot(1, 2, 1)
plt.tricontourf(x_grid, y_grid, v_actual_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Actual u-velocity')
plt.xlabel('x')
plt.ylabel('y')

# Predicted u-velocity
plt.subplot(1, 2, 2)
plt.tricontourf(x_grid, y_grid, v_pred_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Predicted u-velocity')
plt.xlabel('x')
plt.ylabel('y')

plt.tight_layout()
plt.show()


In [ ]:
%matplotlib inline

# Determine the common range for colorbar
vmin = min(v_actual_grid.min(), v_pred_grid.min())
vmax = max(v_actual_grid.max(), v_pred_grid.max())

# Plotting
plt.figure(figsize=(12, 6))

# Actual u-velocity
plt.subplot(1, 2, 1)
plt.tricontourf(x_grid, y_grid, v_actual_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Actual u-velocity')
plt.xlabel('x')
plt.ylabel('y')

# Predicted u-velocity
plt.subplot(1, 2, 2)
plt.tricontourf(x_grid, y_grid, v_pred_grid, levels=50, cmap='jet', vmin=vmin, vmax=vmax)
plt.colorbar(label='u-velocity')
plt.title('Predicted u-velocity')
plt.xlabel('x')
plt.ylabel('y')

plt.tight_layout()
plt.show()
